In [1]:
# Disable info messages from the tensorflow
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
################## Import the libraries #####################
import numpy as np
import pandas as pd
import keras
from keras.models import Model, Sequential
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D,MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler,scale
from sklearn.metrics import accuracy_score
from scipy.ndimage import zoom
import pickle

ModuleNotFoundError: No module named 'keras'

In [2]:
############### Load the dataset ###################
path = 'connect-4.csv'
classes = 3
dataset = np.genfromtxt("https://archive.ics.uci.edu/ml/machine-learning-databases/connect-4/connect-4.data.Z", dtype='str', delimiter=",")
X = dataset[:,0:42]
Y = dataset[:,42]
#Y = LabelEncoder().fit_transform(Y)
#Y = to_categorical(Y,classes)
print(X.shape,Y.shape)

(67557, 42) (67557,)


In [3]:
x = np.zeros(X.shape)

for i, row in enumerate(X):
    for j, col in enumerate(row):
        if col == 'x':
            x[i,j] = 1.0
        if col == 'o':
            x[i,j] = -1.0
        if col == 'b':
            x[i,j] = 0.0

Y = LabelEncoder().fit_transform(Y)
y = to_categorical(Y)
x = MinMaxScaler().fit_transform(x)
print(x.shape,y.shape)

(67557, 42) (67557, 3)


In [4]:
X_train,X_test, Y_train,Y_test = train_test_split(x,y,test_size=0.259884)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(50000, 42) (50000, 3)
(17557, 42) (17557, 3)


In [5]:
########## conversion in Images using Feature Extraction ##############
dim = X_train.shape[1]
x = int(dim**(1/2)) + 1
f = x*x
model = Sequential()
model.add(Dense(f-dim,name='feature', activation='relu',input_shape=(dim,)))
model.add(Dense(classes,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
# here, inputs and labels are same
model.fit(X_train,Y_train,epochs=100,batch_size=128,verbose=0)
extract = Model(model.inputs, model.get_layer('feature').output)
# predict whole inputs through it
x1 = extract.predict(X_train)
x2 = extract.predict(X_test)
# concatenate on horizontal axis
X1 = np.concatenate((X_train, x1), axis=1) 
X2 = np.concatenate((X_test, x2), axis=1) 
print(X1.shape,X2.shape)
train = X1.reshape(X1.shape[0],x+1,x+1)
test = X2.reshape(X2.shape[0],x+1,x+1)
print(train.shape,test.shape)

(50000, 49) (17557, 49)
(50000, 7, 7) (17557, 7, 7)


In [7]:
# RGB Images
from PIL import Image
x_train = []
x_test = []
for i in range(len(train)):
    img = Image.fromarray(train[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    x_train.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
x_train = np.array(x_train,dtype='float32')
for i in range(len(test)):
    img = Image.fromarray(test[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    x_test.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
x_test = np.array(x_test,dtype='float32')
print(x_train.shape,x_test.shape)

(50000, 32, 32, 3) (17557, 32, 32, 3)


In [12]:
############### Classfication ###############
# pretrained model for FE
pretrained_model = keras.applications.DenseNet121(input_shape=(32,32,3), weights=None, include_top=False)
x = Flatten()(pretrained_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
prediction = Dense(3, activation='softmax')(x)
model = Model(inputs=pretrained_model.input, outputs=prediction)
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)
# fit training data and validate on the testing data
model.fit(x_train,Y_train,batch_size=128, epochs=20,verbose=1,validation_data=(x_test,Y_test))

Epoch 1/20
391/391 [==============================] - 23s 58ms/step - loss: 0.5789 - accuracy: 0.7740 - val_loss: 0.6023 - val_accuracy: 0.7531
Epoch 2/20
391/391 [==============================] - 19s 48ms/step - loss: 0.4960 - accuracy: 0.8033 - val_loss: 0.5428 - val_accuracy: 0.8026
Epoch 3/20
391/391 [==============================] - 19s 48ms/step - loss: 0.4615 - accuracy: 0.8173 - val_loss: 0.5085 - val_accuracy: 0.8060
Epoch 4/20
391/391 [==============================] - 19s 48ms/step - loss: 0.4728 - accuracy: 0.8119 - val_loss: 0.4710 - val_accuracy: 0.8182
Epoch 5/20
391/391 [==============================] - 19s 48ms/step - loss: 0.4225 - accuracy: 0.8307 - val_loss: 0.4298 - val_accuracy: 0.8306
Epoch 6/20
391/391 [==============================] - 19s 48ms/step - loss: 0.3975 - accuracy: 0.8399 - val_loss: 0.4209 - val_accuracy: 0.8254
Epoch 7/20
391/391 [==============================] - 19s 49ms/step - loss: 0.3806 - accuracy: 0.8463 - val_loss: 15.9671 - val_accuracy

In [13]:
score = model.evaluate(x_test,Y_test)
acc = model.evaluate(x_train,Y_train)
print('training accuracy: ',acc[1])
print('testing accuracy: ',score[1])

1563/1563 [==============================] - 16s 10ms/step - loss: 0.1585 - accuracy: 0.9376
training accuracy:  0.9375799894332886
testing accuracy:  0.8576066493988037
